In [1]:
text = "I am happy today. I feel sad today."


In [3]:
from textblob import TextBlob
b = TextBlob(text)
b

TextBlob("I am happy today. I feel sad today.")

In [4]:
b.sentences

[Sentence("I am happy today."), Sentence("I feel sad today.")]

In [5]:
b.sentences[0].sentiment

Sentiment(polarity=0.8, subjectivity=1.0)

情感極性0.8，主觀性1.0。說明一下，情感極性的變化範圍是[-1, 1]，-1代表完全負面，1代表完全正面

In [6]:
b.sentences[1].sentiment

Sentiment(polarity=-0.5, subjectivity=1.0)

In [7]:
b.sentiment

Sentiment(polarity=0.15000000000000002, subjectivity=1.0)

In [7]:
text = u"我今天很快樂。我今天很生氣。"
##"很"有影響情緒的百分比

注意在引號前面我們加了一個字母u，它很重要。因為它提示Python，「這一段我們輸入的文本編碼格式是Unicode

In [8]:
from snownlp import SnowNLP
s = SnowNLP(text) ##我要先轉成SNOWNLP的格式才能進行分析

In [9]:
for sentence in s.sentences:
    print(sentence)

我今天很快樂
我今天很生氣


In [10]:
SnowNLP(s.sentences[0]).sentiments

0.9268071116367116

In [11]:
SnowNLP(s.sentences[1]).sentiments

0.39418533974882597

這是因為SnowNLP和textblob的計分方法不同。SnowNLP的情感分析取值，表達的是「這句話代表正面情感的機率」。也就是說，對「我今天很憤怒」一句，SnowNLP認為，它表達正面情感的機率很低很低。

In [12]:
s.sentiments
###整體看準度有待商榷

0.881202463832263

# project comments

In [5]:
from snownlp import SnowNLP
from pymongo import MongoClient
import re

client = MongoClient("mongodb://127.0.0.1:27017")
db = client["<db name>"]
collect = db["<collection name>"]

import pickle
with open("gender", "rb") as f:
    clf1 = pickle.load(f)

with open("vectorizer", "rb") as f1:
    vectorizer = pickle.load(f1)


In [6]:
comments = list(collect.find({},{"_id":0}))

In [7]:
cleaned_comments = []

for comment in comments:
    if len(comment["message"]) != 0:
        comment["score"] = float(format(SnowNLP(SnowNLP(comment["message"]).han).sentiments - 0.5, ".4f"))
    else:
        comment["score"] = None
    
    comment["person_id"] = comment["from"]["id"]
    comment["person_name"] = comment["from"]["name"]
    comment["politician_id"] = comment["post_id"].split("_")[0]
    try:
        name = re.findall('[\u4e00-\u9fa5]+', comment["from"]["name"])[0]
        
        if len(name) == 2:
            comment["gender"] = int(clf1.predict(vectorizer.transform([name[-1]]).toarray())[0])
        else:
            comment["gender"] = int(clf1.predict(vectorizer.transform([name[-2] + " " + name[-1]]).toarray())[0])
    except:
        comment["gender"] = None
    
    del comment["from"]
    
    cleaned_comments.append(comment)

In [8]:
db1 = client["project"]
db1["comments"].insert_many(cleaned_comments)